In [ ]:
import requests
import random
import time
import os
from concurrent.futures import ThreadPoolExecutor

In [ ]:
BASE_URL = "https://www.kavak.com/mx/seminuevos"
PATH_TO_SAVE = '../data/raw_html/'
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'es-MX,es;q=0.9'
}
os.makedirs(PATH_TO_SAVE, exist_ok=True)

In [ ]:
def get_raw_data(url, page, path):
    try:
        time.sleep(random.uniform(1, 3))
        response = requests.get(f"{url}?page={page}", headers=HEADERS, timeout=20)
        
        if response.status_code == 200:
            print(f"Pagina {page} descargada correctamente. ")
            filename = f"{path}kavak_page_{page}.html"
            with open(filename, 'w', encoding='utf-8') as f:
                f.write(response.text)
            
        else:
            print(f"Warning: La pagina {page} devolvio status {response.status_code}")
            return None
        
    except Exception as e:
        print(f"Sucedio un error {e} tratando de descargar la pagina numero: {page}")    
        return None

In [ ]:
def save_html(html_content, path, page):
    filename = f"{path}pagina_{page}.html"
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(html_content.text)

In [ ]:
def main(pages_limit=3):
    print(f"Iniciando crawler secuencial para {pages_limit} páginas...")
    futures = []
    with ThreadPoolExecutor(max_workers=3) as executor:
        for page in range(1, pages_limit + 1):
            future = executor.submit(get_raw_data, BASE_URL, page, PATH_TO_SAVE)
            futures.append(future)
    
    for future in futures:
        #.result() BLOQUEA hasta que este hilo especifico termine
        # y devuelve lo que hizo el 'return' de get_raw_data
        try:
            response = future.result()
        except Exception as e:
            print(f"Hilo fallido: {e}")

In [ ]:
if __name__ == "__main__":
    main(pages_limit=3)